In [1]:
import pandas, pickle

# Resut Metrics:

In [ ]:
Recall = correctmatch/number of name
Precision = correctmatch / number of retrieved
F-score = 2 * (precision * recall)/(precision + recall)

In [29]:
def recall(df): 
    correct_count = len(new_df.loc[new_df["HumanEval"] == "YES"])

    r = correct_count/len(df)

    print(f"Total query: {len(df)} \nCorrect correspondence count: {correct_count}")
    print(f"Recall: {r}")
    print(f"\n")

    return r

In [30]:
def precision(df):
    total_retrieved = 0
    correct_count = len(new_df.loc[new_df["HumanEval"] == "YES"])

    for index, row in df.iterrows():
        # if retrieved_names is empty --> skip count
        if row['RetrievedNames'] == "[]":
            continue
        for target in row['RetrievedNames']:
            total_retrieved += 1

    try:
        p = correct_count / total_retrieved
    except ZeroDivisionError:
        print(f"The total retrieved is: {total_retrieved}!!!!")

    print(f"Total query: {len(df)}\nTotal retrieved: {total_retrieved}\nCorrect correspondence count: {correct_count} ")
    print(f"Precision: {p}")
    print(f"\n")

    return p


In [33]:
def f_measure(df):
    try:
        p = precision(df)
        r = recall(df)
        f = 2 * (p * r)/(p + r)
        print(f"F-measure: {f}\n")
        return f
    except ZeroDivisionError:
        print(f"Both precision is {precision} and recall is {recall}!!!")
        return 0

# Exact string matching

In [7]:
df = pandas.read_pickle("results/bronbeekToNmvwExactMatchResults.pkl")
len(df.loc[df["MATCH"] == "YES"])

0

Interesting; There is no exact match of names in two dataset 

# Surname Matching

In [8]:
df = pandas.read_pickle("results/bronbeekToNmvwSurnameMatchResults.pkl")
len(df.loc[df["MATCH"] == "YES"])

6495

In [14]:
df.columns

Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2',
       'Unnamed: 0.1', 'Unnamed: 0', 'ConstituentID', 'ConstituentTypeID',
       'Active', 'AlphaSort', 'LastName', 'FirstName', 'NameTitle',
       'Institution', 'DisplayName', 'BeginDate', 'EndDate', 'DisplayDate',
       'Biography', 'Code', 'Nationality', 'School', 'LoginID', 'EnteredDate',
       'SysTimeStamp', 'PublicAccessOld', 'Remarks', 'Position', 'MiddleName',
       'Suffix', 'CultureGroup', 'salutation', 'Approved', 'PublicAccess',
       'IsPrivate', 'DefaultNameID', 'SystemFlag', 'InternalStatus',
       'DefaultDisplayBioID', 'BeginDateISO', 'EndDateISO', 'FullName',
       'RetrievedNames', 'MATCH'],
      dtype='object')

In [17]:
new_df = df.loc[df["MATCH"] == "YES"][['AlphaSort', 'FirstName', 'NameTitle', 'Biography', 'LastName', 'DisplayName', 'RetrievedNames', 'MATCH']].sample(n = 60, random_state=1)

In [18]:
# new_df.to_csv("results/SurnameMatchResult_sample_60.csv")

After that we manually annotate true, false and unknown correspondance.

In [13]:
new_df = pandas.read_csv("results/SurnameMatchResult_sample_60.csv", sep=";")

In [17]:
true_correspondance = len(new_df.loc[new_df["HumanEval"] == "YES"])
false_correspondance = len(new_df.loc[new_df["HumanEval"] == "NO"])
unknown_correspondance = len(new_df.loc[new_df["HumanEval"] == "?"])

print(f"Number of correct correspondance: {true_correspondance} \nNumber of correct correspondance: {false_correspondance} \nNumber of correct correspondance: {unknown_correspondance} \nout of randomly chosen 60 sample!!!")

Number of correct correspondance: 13 
Number of correct correspondance: 23 
Number of correct correspondance: 24 
out of randomly chosen 60 sample!!!


In [34]:
f_measure(new_df)

Total query: 60
Total retrieved: 6200
Correct correspondence count: 13 
Precision: 0.002096774193548387


Total query: 60 
Correct correspondence count: 13
Recall: 0.21666666666666667


F-measure: 0.004153354632587859



0.004153354632587859

# Abbreviation Matching

In [21]:
df = pandas.read_pickle("results/bronbeekToNmvwAbbreviationMatchResults.pkl")
# The abbreviation match is done based on left outer join, 
# so it only contains rows when there was a positive match on the constructed abbrv.
len(df.index)

1177

In [22]:
df.columns

Index(['nmvw_uri', 'name_label', 'Abbreviations', 'Unnamed: 0.5',
       'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'ConstituentID', 'ConstituentTypeID', 'Active',
       'AlphaSort', 'LastName', 'FirstName', 'NameTitle', 'Institution',
       'DisplayName', 'BeginDate', 'EndDate', 'DisplayDate', 'Biography',
       'Code', 'Nationality', 'School', 'LoginID', 'EnteredDate',
       'SysTimeStamp', 'PublicAccessOld', 'Remarks', 'Position', 'MiddleName',
       'Suffix', 'CultureGroup', 'salutation', 'Approved', 'PublicAccess',
       'IsPrivate', 'DefaultNameID', 'SystemFlag', 'InternalStatus',
       'DefaultDisplayBioID', 'BeginDateISO', 'EndDateISO', 'FullName'],
      dtype='object')

In [24]:
new_df = df[['nmvw_uri', 'name_label', 'Abbreviations', 'FirstName', 'LastName', 'DisplayName']].sample(n = 60, random_state=1)

In [25]:
# new_df.to_csv("results/AbbreviationMatchResult_sample_60.csv")

Implementation Issue: When there was already abbreviation in the name, we only took the first letter.

Reflexion: 
    - It is quite difficult to disambiguate based on the abbreviation of names
    - also sometime there is lastname first then the abbreviation is mention, so hard to find out which one is supposed to be the last name.
    - For example, if there is a name "s Tanaka" and other data source mentions S. Tanaka, I cannot deterministicly say that they are taking about the same person.
    - It is difficult to say negative or positive match based on this limited information. 
    

In [36]:
new_df = pandas.read_csv("results/AbbreviationMatchResult_sample_60.csv", sep=";")

In [28]:
true_correspondance = len(new_df.loc[new_df["HumanEval"] == "YES"])
false_correspondance = len(new_df.loc[new_df["HumanEval"] == "NO"])
unknown_correspondance = len(new_df.loc[new_df["HumanEval"] == "?"])

print(f"Number of correct correspondance: {true_correspondance} \nNumber of correct correspondance: {false_correspondance} \nNumber of correct correspondance: {unknown_correspondance} \nout of randomly chosen 60 sample!!!")

Number of correct correspondance: 8 
Number of correct correspondance: 7 
Number of correct correspondance: 45 
out of randomly chosen 60 sample!!!


- The reason for a lot of unknown correspondance is that there is too little information to disabmiguate. 
- If there is a positive match on two dataset based on abbreviation, it is likely that that is basis to consider that both of the entity might be reffereing to the same real-world person. 
- However, based on the abbreaviated name it is not possible to establish that they are exactly same person. Most of the cases the full name is either not available or the partial of the abbreviated matches. 
- So, from human evaluation we can say it is likely or unlikely match, most of the time we cannot accept or deny the correspondance. 
_ In conclusion, we need more information beyond the name. 

In [37]:
f_measure(new_df)

KeyError: 'RetrievedNames'

# Fuzzy String Match

In [3]:
df = pandas.read_pickle("results/bronbeekToNmvwFuzzyStringMatchResults.pkl")
len(df.loc[df["MATCH"] == "NO"])

19

It is interesting that ONLY 19 did not have any match.

In [4]:
len(df.loc[df["MATCH"] == "YES"])

15363

In [5]:
df.columns

Index(['Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2',
       'Unnamed: 0.1', 'Unnamed: 0', 'ConstituentID', 'ConstituentTypeID',
       'Active', 'AlphaSort', 'LastName', 'FirstName', 'NameTitle',
       'Institution', 'DisplayName', 'BeginDate', 'EndDate', 'DisplayDate',
       'Biography', 'Code', 'Nationality', 'School', 'LoginID', 'EnteredDate',
       'SysTimeStamp', 'PublicAccessOld', 'Remarks', 'Position', 'MiddleName',
       'Suffix', 'CultureGroup', 'salutation', 'Approved', 'PublicAccess',
       'IsPrivate', 'DefaultNameID', 'SystemFlag', 'InternalStatus',
       'DefaultDisplayBioID', 'BeginDateISO', 'EndDateISO', 'FullName',
       'RetrievedNames', 'MATCH'],
      dtype='object')

In [8]:
new_df = df.loc[df["MATCH"] == "YES"][['AlphaSort', 'FirstName', 'MiddleName', 'LastName', 'DisplayName', 'FullName', 'Suffix', 'RetrievedNames', 'MATCH']].sample(n = 60, random_state=1)

In [9]:
new_df.to_csv("results/FuzzyStringMatchResult_sample_60.csv")

DO: Manual Evaluation

Implementation details: 
    - max three candidate correspondance for each row based on edit distance < 75
    - There was some name labels with 1-4 character which does not contain much sense
    - And they are contributing to high number of Edit distance match
    - I had to re-run the code living out the candidate names that have less than 5 character in it.

For fuzzy string matching, all the character that are wrongly encoded creating a problem.

In [41]:
new_df = pandas.read_csv("results/FuzzyStringMatchResult_sample_60.csv", sep=";")

In [43]:
true_correspondance = len(new_df.loc[new_df["HumanEval"] == "YES"])
false_correspondance = len(new_df.loc[new_df["HumanEval"] == "NO"])
unknown_correspondance = len(new_df.loc[new_df["HumanEval"] == "?"])

print(f"Number of correct correspondance: {true_correspondance} \nNumber of correct correspondance: {false_correspondance} \nNumber of correct correspondance: {unknown_correspondance} \nout of randomly chosen 60 sample!!!")

Number of correct correspondance: 5 
Number of correct correspondance: 38 
Number of correct correspondance: 17 
out of randomly chosen 60 sample!!!


In [42]:
f_measure(new_df)

Total query: 60
Total retrieved: 3588
Correct correspondence count: 5 
Precision: 0.0013935340022296545


Total query: 60 
Correct correspondence count: 5
Recall: 0.08333333333333333


F-measure: 0.002741228070175439



0.002741228070175439

### Data Exploration

In [56]:
df1 = pandas.read_pickle("../nmvw_data/ccrdfconst/person_names.pkl")
list_of_names = [str(row['name_label']) for i, row in df1.iterrows() if len(str(row['name_label']))>4]

In [57]:
len(list_of_names)

39336

In [37]:
list_of_names

['Ph. Curtin',
 'Albrecht, H.',
 'Marie Yvonne Curtis',
 'J.L. Curtis',
 'Anne van Cutsem',
 'Cushing, F.H.',
 'Sophie Curtil',
 'F.H. Cushing',
 'Edward S. Curtis',
 'Curtis, E.S.',
 'H.C. Cutler',
 'B. Cuthbertson',
 'E. de Cuyper',
 'W.F. Albright',
 'Annette Czekelius',
 'cycloop',
 'A. Czekanowska',
 'Mw.  Cutter',
 'T.A.S. Cuttack',
 'D. Czarkowska Starzecka',
 'C.B. Czitrom',
 'J. Czekanowski',
 'P. (Paul) Cuypers',
 'C.H. Cutter',
 'Jan-B. Cuypers',
 'C.H. Cutter',
 'van Cuyk',
 'E. van Cutsem',
 'H. Albrecht',
 'J.M.C. D Azevedo',
 'Dino',
 'M.-E. D Udekem',
 'O. de Albuquerque',
 "Anne D'Alleva",
 'M. D Hertefelt',
 'Mark (assistant) D Amato',
 'Warren D azevedo',
 'A. D Alpoim Guedes',
 "A.R. d' Huy",
 'W.L. D Azevedo',
 "Ada d'Aloja",
 'J.P. D Assac',
 'G. D Ieteren',
 'H. D Estre',
 'Pangeran Djatikoesoemo',
 'P. D Ieteren',
 "Ch.J. d' Ancona",
 'Da Costa, dr. J.F.',
 'Ibrahim Mohammed Al-BuwÃ¡rdi',
 'Nilton Moreira da Silva',
 'J.D. von Miller',
 'A. Da Cruz',
 'Lherisson

In [41]:
for name in list_of_names:
    if len(name)<3: 
        print(name)

V.
II
In
NN
nn
Ch


In [52]:
df1.loc[df1['name_label'].str.len() < 4]

,nmvw_uri,name_label
149,https://hdl.handle.net/20.500.11840/pi41248,Dam
542,https://hdl.handle.net/20.500.11840/pi41268,Dee
799,https://hdl.handle.net/20.500.11840/pi49444,Des
925,https://hdl.handle.net/20.500.11840/pi62661,DFP
2050,https://hdl.handle.net/20.500.11840/pi49570,dwm
2140,https://hdl.handle.net/20.500.11840/pi46889,Ebo
4857,https://hdl.handle.net/20.500.11840/pi62353,Ank
6899,https://hdl.handle.net/20.500.11840/pi81818,Aqo
8916,https://hdl.handle.net/20.500.11840/pi47327,V.
9313,https://hdl.handle.net/20.500.11840/pi48471,Art


In [38]:
new_df = pandas.read_csv("results/AbbreviationMatchResult_sample_60.csv", sep=";")